In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
# Download YOLOv7 repository and install requirements

%cd /content
!git clone https://github.com/WongKinYiu/yolov7.git
%cd yolov7
!pip install -r requirements.txt

/content
Cloning into 'yolov7'...
remote: Enumerating objects: 1094, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 1094 (delta 0), reused 1 (delta 0), pack-reused 1091
Receiving objects: 100% (1094/1094), 69.85 MiB | 34.16 MiB/s, done.
Resolving deltas: 100% (523/523), done.
/content/yolov7
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 4.9 MB/s 


In [3]:
!sed -n -e 1557p -e 1404p /content/yolov7/utils/loss.py

            from_which_layer = from_which_layer[fg_mask_inboxes]
            from_which_layer = from_which_layer[fg_mask_inboxes]


In [4]:
# 注意跳脫自字元
!sed -i '1557s/from_which_layer\[fg_mask_inboxes\]/from_which_layer.to(fg_mask_inboxes.device)\[fg_mask_inboxes\]/' /content/yolov7/utils/loss.py
!sed -i '1404s/from_which_layer\[fg_mask_inboxes\]/from_which_layer.to(fg_mask_inboxes.device)\[fg_mask_inboxes\]/' /content/yolov7/utils/loss.py

!sed -n -e 1557p -e 1404p /content/yolov7/utils/loss.py

            from_which_layer = from_which_layer.to(fg_mask_inboxes.device)[fg_mask_inboxes]
            from_which_layer = from_which_layer.to(fg_mask_inboxes.device)[fg_mask_inboxes]


In [5]:
import os
%cd /content/yolov7

# 無人機dataset
'''
!pip install -q --upgrade gdown
!gdown --id '1HIC6u_mBvkxdR8F9-sTlKkLXeAhyzC1G'
!unzip -q VisDrone4.zip
os.remove("VisDrone4.zip")
'''

!cp '/gdrive/MyDrive/Alex_Lin/無人機比賽/VisDrone4.zip' '/content/yolov7/VisDrone4.zip'
!unzip -q VisDrone4.zip
os.remove("VisDrone4.zip")

/content/yolov7


In [6]:
yaml_file = "/content/yolov7/VisDrone4/data.yaml"
if os.path.exists(yaml_file) == False:
  f= open(yaml_file,"w+")
  f.write("train: /content/yolov7/VisDrone4/train/images\n") # train資料位置
  f.write("val: /content/yolov7/VisDrone4/val/images\n")
  f.write("nc: 4\n") # 修改:有幾個類別
  f.write("\n")
  # 自己dataset的類別
  f.write("names: ['car', 'hov', 'person', 'motorcycle']\n") # 修改
  f.close()


In [7]:
%cd /content/yolov7

/content/yolov7


In [8]:
!pwd

/content/yolov7


下載前一階段在無人機訓練4類別epoch的權重檔準備接類訓練

In [9]:
''' gdown 常常很奇怪
!gdown --id '1FWlpxiI-ai1GmSMBk_zqKfRwQD3un-0H' --output data.zip
!unzip -q data.zip
os.remove("data.zip")
'''

!cp '/gdrive/MyDrive/Alex_Lin/無人機比賽/用到VisDrone/2_best.zip' '/content/yolov7/2_best.zip'
!unzip -q 2_best.zip
os.remove("2_best.zip")

換成VisDrone dataset的Anchor box尺寸

In [10]:
!sed -n -e 8p -e 9p -e 10p -e 11p cfg/training/yolov7-w6.yaml

  - [ 19,27,  44,40,  38,94 ]  # P3/8
  - [ 96,68,  86,152,  180,137 ]  # P4/16
  - [ 140,301,  303,264,  238,542 ]  # P5/32
  - [ 436,615,  739,380,  925,792 ]  # P6/64


In [11]:
# 修改這幾行內容 注意跳脫字元
!sed -i '8s/19,27,  44,40,  38,94/4,9, 5,16, 6,25/' cfg/training/yolov7-w6.yaml
!sed -i '9s/96,68,  86,152,  180,137/11,14, 10,35, 16,27/' cfg/training/yolov7-w6.yaml
!sed -i '10s/140,301,  303,264,  238,542/25,19, 19,51, 37,35/' cfg/training/yolov7-w6.yaml
!sed -i '11s/436,615,  739,380,  925,792/34,82, 64,58, 97,130/' cfg/training/yolov7-w6.yaml

!sed -n -e 8p -e 9p -e 10p -e 11p cfg/training/yolov7-w6.yaml

  - [ 4,9, 5,16, 6,25 ]  # P3/8
  - [ 11,14, 10,35, 16,27 ]  # P4/16
  - [ 25,19, 19,51, 37,35 ]  # P5/32
  - [ 34,82, 64,58, 97,130 ]  # P6/64


### 使用 wandb 記錄訓練過程
- 先官網登入：https://wandb.ai/site 
- 右上方頭像 -> settings -> 複製 API keys -> 填入下方輸入處

In [12]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.9 MB 4.8 MB/s 
     |████████████████████████████████| 168 kB 74.7 MB/s 
     |████████████████████████████████| 182 kB 69.6 MB/s 
     |████████████████████████████████| 62 kB 1.9 MB/s 
     |████████████████████████████████| 168 kB 68.5 MB/s 
     |████████████████████████████████| 166 kB 60.6 MB/s 
     |████████████████████████████████| 166 kB 56.5 MB/s 
     |████████████████████████████████| 162 kB 62.6 MB/s 
     |████████████████████████████████| 162 kB 75.8 MB/s 
     |████████████████████████████████| 158 kB 72.8 MB/s 
     |████████████████████████████████| 157 kB 77.7 MB/s 
     |████████████████████████████████| 157 kB 80.6 MB/s 
     |████████████████████████████████| 157 kB 88.3 MB/s 
     |████████████████████████████████| 157 kB 60.4 MB/s 
     |████████████████████████████████| 157 kB 86.3 MB/s 
     |████████████████████████████

In [13]:
import wandb
# Logging
id = wandb.util.generate_id()
wandb_run = wandb.init()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [14]:
id 

'2g0g4su6'

In [ ]:
# 注意--data和--weights的路徑

!python train_aux.py --epochs 60 \
--batch-size 22 \
--data VisDrone4/data.yaml \
--img 1280 1280 \
--device 0 \
--cfg cfg/training/yolov7-w6.yaml \
--weights '2_best.pt' \
--hyp data/hyp.scratch.custom.yaml

# A100、1280、BS22 大約一個epoch 6分12秒

YOLOR 🚀 v0.1-104-g941b94c torch 1.12.1+cu113 CUDA:0 (A100-SXM4-40GB, 40536.1875MB)

Namespace(adam=False, artifact_alias='latest', batch_size=22, bbox_interval=-1, bucket='', cache_images=False, cfg='cfg/training/yolov7-w6.yaml', data='VisDrone4/data.yaml', device='0', entity=None, epochs=60, evolve=False, exist_ok=False, global_rank=-1, hyp='data/hyp.scratch.custom.yaml', image_weights=False, img_size=[1280, 1280], label_smoothing=0.0, linear_lr=False, local_rank=-1, multi_scale=False, name='exp', noautoanchor=False, nosave=False, notest=False, project='runs/train', quad=False, rect=False, resume=False, save_dir='runs/train/exp', save_period=-1, single_cls=False, sync_bn=False, total_batch_size=22, upload_dataset=False, weights='2_best.pt', workers=8, world_size=1)
tensorboard: Start with 'tensorboard --logdir runs/train', view at http://localhost:6006/
2022-12-03 19:45:44.100146: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numer

都跑完時到側邊欄下載權重檔